In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive to access files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Setup and Import Libraries**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


**Set Directory Paths**

In [ ]:
base_dir = '/content/drive/MyDrive/Data/weather classification'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


**Image Data Generators**

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 400 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
Found 525 images belonging to 4 classes.


**Building the CNN Model**

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  # 4 classes
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


**Train the Model**

In [44]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.8354 - loss: 0.4422 - val_accuracy: 0.8150 - val_loss: 0.5269
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.9061 - loss: 0.2651 - val_accuracy: 0.7500 - val_loss: 0.6952
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.8841 - loss: 0.2389 - val_accuracy: 0.8000 - val_loss: 0.6937
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.9147 - loss: 0.2072 - val_accuracy: 0.7950 - val_loss: 0.6295


**Evaluate on Test Set**

In [45]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 456ms/step - accuracy: 0.8711 - loss: 0.5234
Test Accuracy: 87.05%


**Create Augmented Image Data Generator**

In [ ]:
train_datagen_augmented = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator_augmented = train_datagen_augmented.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 400 images belonging to 4 classes.


**Train the Augmented Model**

In [46]:
history_augmented = model.fit(
    train_generator_augmented,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.8286 - loss: 0.4456 - val_accuracy: 0.8150 - val_loss: 0.5150
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.8667 - loss: 0.4194 - val_accuracy: 0.7700 - val_loss: 0.7051
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.8734 - loss: 0.3283 - val_accuracy: 0.7750 - val_loss: 0.7606
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.8564 - loss: 0.3340 - val_accuracy: 0.8250 - val_loss: 0.4866
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.8714 - loss: 0.3442 - val_accuracy: 0.7400 - val_loss: 0.9618
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8518 - loss: 0.3241 - val_accuracy: 0.7500 - val_loss: 0.7206
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8940 - loss: 0.2781 - val_accuracy: 0.5950 - val_loss: 1.6829


**Evaluate on Test Set**

In [47]:
test_loss_augmented, test_accuracy_augmented = model.evaluate(test_generator)
print(f"Test Accuracy after Augmentation: {test_accuracy_augmented * 100:.2f}%")


17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 515ms/step - accuracy: 0.9347 - loss: 0.2540
Test Accuracy after Augmentation: 92.00%


**Pre-trained Network for Feature Extraction**:
**Import Pre-trained Model**

In [48]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
base_model.trainable = False  # Freeze base model layers

model_pretrained = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(4, activation='softmax')  # 4 classes
])

model_pretrained.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


**Train the Pre-trained Model**

In [51]:
history_pretrained = model_pretrained.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


Epoch 1/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 152s 12s/step - accuracy: 0.6426 - loss: 1.1845 - val_accuracy: 0.6750 - val_loss: 1.1270
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 234s 14s/step - accuracy: 0.7648 - loss: 1.0990 - val_accuracy: 0.7250 - val_loss: 1.0600
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 153s 12s/step - accuracy: 0.8186 - loss: 1.0233 - val_accuracy: 0.7400 - val_loss: 1.0106


**Evaluate on Test Set**

In [53]:
test_loss_pretrained, test_accuracy_pretrained = model_pretrained.evaluate(test_generator)
print(f"Test Accuracy with Pre-trained Network: {test_accuracy_pretrained * 100:.2f}%")


17/17 ━━━━━━━━━━━━━━━━━━━━ 129s 8s/step - accuracy: 0.6774 - loss: 1.1700
Test Accuracy with Pre-trained Network: 69.14%
